In [1]:
import builtins

from coppertop.pipe import *

from dm.core.types import matrix, vec, pytuple, t, dseq, pylist, num, offset, bool
from bones.core.sentinels import Missing
from bones.lang.structs import tv
from bones.lang.utils import Constructors
from bones.lang.metatypes import BTNom
from dm.testing import check, equals
from dm.pp import PP
from _ import *



@coppertop
def sequence_(n):
    return range(n)
offset.sequence_ = sequence_


# implement matrices as dseq of dseq

# are vec and matrix different types?
#
# against
# * takeCol -> vec, takeCols -> matrix, then need two functions for A >> mmul >> B >> (takeCols >> [0]) and A >> mmul >> B >> (takeCol >> 1) though 
#   conceptually they mean the same thing
# * we would like to render the type correctly to match our intuition - we can't tell which way to render a vec (col or row) can we default to col?
# * doubles up number of methods for each type - then also need tensor, ndarray shares functionality
#
# for
# if we want to access a vec - we don't need to know if it is a column or row - at, atput, count
#
# start off with essential relationships rather than inner qualities, i.e. how is it used versus some conception of what it is

mutable = BTNom.ensure('mutable')
maddResult = BTNom.ensure('maddResult')
rowmajor = BTNom.ensure('rowmajor')
Matrix = (matrix&rowmajor&dseq).nameAs('Matrix')
       
@coppertop
def _newMatrix(t:Constructors, m:t.count, n:t.count) -> Matrix:
    answer = [Missing] * m
    for i in m >> offset.sequence_:
        answer[i] = [0] * n
    return dseq(Matrix, answer)

@coppertop
def _newMatrix(t:Constructors, contents:pylist) -> Matrix:
    # could check it's rectangular
    return dseq(Matrix, contents)

Matrix.setConstructor(_newMatrix)


@coppertop(style=unary)
def toMatrix(lol:pylist):
    return Matrix(lol)

@coppertop(style=unary)
def toCol(l:pylist):
    m = l >> count
    answer = Matrix(m, 1)
    for i in m >> offset.sequence_:
        answer[i][0] = l[i]
    return answer

@coppertop(style=unary)
def toRow(l:pylist):
    n = l >> count
    answer = Matrix(1, n)
    for j in n >> offset.sequence_:
        answer[0][j] = l[j]
    return answer

@coppertop(style=unary)
def toSeq(l:pylist):
    raise NotYetImplemented()

@coppertop(style=unary)
def toSeqOfSeq(A:Matrix):
    raise NotYetImplemented()

@coppertop(style=unary)
def shape(A:Matrix) -> pytuple:
    return len(A), len(A[0])

@coppertop(style=unary)
def min(A:Matrix) -> num:
    return min([min(r) for r in A])

@coppertop(style=unary)
def max(A:Matrix) -> num:
    return max([max(r) for r in A])

@coppertop(style=unary)
def minmax(A:Matrix) -> pytuple:
    return min([min(r) for r in A]), max([max(r) for r in A])

@coppertop(style=unary)
def abs(A:Matrix) -> Matrix:
    m, n = A >> shape
    answer = Matrix(m, n)
    for i in m >> offset.sequence_:
        for j in n >> offset.sequence_:
            answer[i][j] = builtins.abs(A[i][j])
    return answer

@coppertop(style=unary)
def T(A:Matrix) -> Matrix:
    m, n = A >> shape
    answer = Matrix(n, m)
    for i in m >> offset.sequence_:
        for j in n >> offset.sequence_:
            answer[j][i] = A[i][j]
    return answer

@coppertop
def PP(A:Matrix):
    # 'show 2 figures of smallest'
    m, n = A >> shape
    a = A >> abs
    minA, maxA = minmax(A)
    numPreDot = format(maxA, '.0f') >> count
    numPostDot = 0
    numForDot = 0
    padding = numPreDot + numForDot + numPostDot
    for i in m >> offset.sequence_:
        line = ''
        for j in n >> offset.sequence_:
            line += ('  ' if line else '') + builtins.format(A[i][j], '.0f') >> pad(_, dict(right=padding))
        line >> PP
    return A

@coppertop(style=binary)
def madd(A:Matrix, B:Matrix) -> Matrix:
    mA, nA = A >> shape
    mB, nB = B >> shape
    assert mA == mB and nA == nB
    answer = Matrix(mA, nA)
    for i in mA >> offset.sequence_:
        for j in nA >> offset.sequence_:
            answer[i][j] = A[i][j] + B[i][j]
    return answer

@coppertop(style=binary)
def madd(A:Matrix&mutable, B:Matrix) -> Matrix&mutable:
    mA, nA = A >> shape
    mB, nB = B >> shape
    assert mA == mB and nA == nB
    for i in mA >> offset.sequence_:
        for j in nA >> offset.sequence_:
            A[i][j] = A[i][j] + B[i][j]
    return A

@coppertop(style=unary)
def XXT(A:Matrix) -> Matrix:
    return A >> mmul >> (A >> T)

@coppertop(style=unary)
def XTX(A:Matrix) -> Matrix:
    return A >> T >> mmul >> A

@coppertop(style=binary)
def mmul(A:Matrix, B:Matrix) -> Matrix:
    m, n = A >> shape
    mB, nB = B >> shape
    assert n == mB
    answer = Matrix(m, nB)
    for i in m >> offset.sequence_:
        for j in nB >> offset.sequence_:
            for k in n >> offset.sequence_:
                answer[i][j] += (A[i][k]) * (B[k][j])
    return answer

@coppertop(style=binary)
def mtimes(s:num, A:Matrix) -> Matrix:
    m, n = A >> shape
    answer = Matrix(m, n)
    for i in m >> offset.sequence_:
        for j in n >> offset.sequence_:
            answer[i][j] = A[i][j] * s
    return answer

@coppertop(style=binary)
def mtimes(A:Matrix, s:num) -> Matrix:
    m, n = A >> shape
    answer = Matrix(m, n)
    for i in m >> offset.sequence_:
        for j in n >> offset.sequence_:
            answer[i][j] = A[i][j] * s
    return answer

@coppertop(style=binary)
def scalarProductRC(row:Matrix, col:Matrix) -> num:
    mR, nR = row >> shape
    mC, nC = col >> shape
    assert row >> isRow
    assert col >> isCol
    answer = 0
    for i in nR >> offset.sequence_:
        answer += row[0][i] * col[i][0]
    return answer

@coppertop(style=binary)
def outerProductCR(col:Matrix, row:Matrix) -> Matrix:
    mC, nC = col >> shape
    mR, nR = row >> shape
    assert col >> isCol
    assert row >> isRow
    answer = Matrix(mC, nR)
    for j in nR >> offset.sequence_:
        for i in mC >> offset.sequence_:
            answer[i][j] += row[0][j] * col[i][0]
    return answer

@coppertop(style=binary)
def outerProductCR_(col:Matrix, row:Matrix) -> (Matrix&mutable&maddResult)^Matrix&mutable&maddResult:
    return outerProductCR_impl(_, col, row)

@coppertop(style=unary)
def outerProductCR_impl(answer: Matrix&mutable&maddResult, col:Matrix, row:Matrix) -> Matrix&mutable&maddResult:
    mC, nC = col >> shape
    mR, nR = row >> shape
    assert col >> isCol
    assert row >> isRow
    for j in nR >> offset.sequence_:
        for i in mC >> offset.sequence_:
            answer[i][j] += row[0][j] * col[i][0]
    return answer

@coppertop(style=binary)
def madd(A:Matrix&mutable&maddResult, fnB:((Matrix&mutable&maddResult)^(Matrix&mutable&maddResult))) -> Matrix&mutable&maddResult:
    return fnB(A)

@coppertop(style=binary)
def atRow(A:Matrix, o:offset) -> Matrix:
    m, n = A >> shape
    answer = Matrix(1, n)
    for j in n >> offset.sequence_:
        answer[0][j] = A[o][j]
    return answer    

@coppertop(style=binary)
def atCol(A:Matrix, o:offset) -> Matrix:
    m, n = A >> shape
    answer = Matrix(m, 1)
    for i in m >> offset.sequence_:
        answer[i][0] = A[i][o]
    return answer

@coppertop(style=unary)
def isCol(A:Matrix) -> bool:
    m, n = A >> shape
    return n == 1

@coppertop(style=unary)
def isRow(A:Matrix) -> bool:
    m, n = A >> shape
    return m == 1



<br>

### matrix times vector

For $A x$, normally is thought of as the scalar (aka inner or dot) product of each row in $A$ with $x$, e.g. 

$$
\begin{align}
\begin{pmatrix}
2 & 3 \\
2 & 4 \\
3 & 7
\end{pmatrix}
\begin{pmatrix}
1 \\
2 \\
\end{pmatrix}
=
\begin{pmatrix}
( 2, 3 ) \cdot (1, 2) \\
( 2, 4 ) \cdot (1, 2) \\
( 3, 7 ) \cdot (1, 2)
\end{pmatrix}
\end{align}
$$


In [2]:
A = [[2,3],[ 2,4], [3,7]] >> toMatrix >> PP
'' >> PP
x = [1, 2] >> toCol >> PP

  2  3
  2  4
  3  7

  1
  2


ai has found even more efficient ways than the high school way - https://arstechnica.com/information-technology/2022/10/deepmind-breaks-50-year-math-record-using-ai-new-record-falls-a-week-later/

In [3]:
# @coppertop(style=binary, module='notes.impl')
@coppertop(style=binary)
def mulMatVec1(A:Matrix, v:Matrix) -> Matrix:
    mA, nA = A >> shape
    mV, nV = v >> shape
    assert nA == mV
    assert v >> isCol
    answer = Matrix(mA, nV)
    for i in mA >> offset.sequence_:
        for j in nV >> offset.sequence_:
            answer[i][j] = (A >> atRow >> i) >> scalarProductRC >> (v >> atCol >> j)
    return answer

In [4]:
A >> mulMatVec1 >> x >> PP;

  8
 10
 17


More intuitively we can think of the whole answer as the weighted columns of $A$ ($\{\textbf{a}_1, \textbf{a}_2\}$) with the weights coming from $x$, thus:

$$
\begin{align}
\begin{pmatrix}
2 & 3 \\
2 & 4 \\
3 & 7
\end{pmatrix}
\begin{pmatrix}
1 \\
2 \\
\end{pmatrix}
= 1 \cdot
\begin{pmatrix}
2\\
2\\
3
\end{pmatrix}
+ 2 \cdot
\begin{pmatrix}
2\\
4\\
7
\end{pmatrix}
\end{align}
$$

In [5]:
# @coppertop(style=binary, module='notes.intuitive')
@coppertop(style=binary)
def mulMatVec2(A:Matrix, v:Matrix) -> Matrix:
    mA, nA = A >> shape
    mV, nV = v >> shape
    assert nA == mV
    assert v >> isCol
    answer = Matrix(mA, nV)
    for j in nA >> offset.sequence_:
        scalar = v[j][0]
        answer = answer >> madd >> (A >> atCol >> j >> mtimes >> scalar)
    return answer

In [6]:
A >> mulMatVec2 >> x >> PP;

  8
 10
 17


<br>

### matrix times matrix

In [7]:
B = [[3,4,6], [4,5,6]] >> toMatrix
A >> mmul >> B >> PP;

 18  23  30
 22  28  36
 37  47  60


In [8]:
# @coppertop(style=binary, module='notes.intuitive')
@coppertop(style=binary)
def mulMatMat3(A:Matrix, B:Matrix) -> Matrix:
    mA, nA = A >> shape
    mB, nB = B >> shape
    assert nA == mB
    answer = Matrix(mA, nB)
    for i in nA >> offset.sequence_:
        'outer:' >> PP
        outerProduct = (A >> atCol >> i) >> outerProductCR >> (B >> atRow >> i) >> PP
        'running:' >> PP
        answer = answer >> madd >> outerProduct >> PP
        
    return answer

In [9]:
A >> PP
'' >> PP
B >> PP
'-----' >> PP
A >> mulMatMat3 >> B;

  2  3
  2  4
  3  7

  3  4  6
  4  5  6
-----
outer:
  6  8  12
  6  8  12
  9  12  18
running:
  6  8  12
  6  8  12
  9  12  18
outer:
 12  15  18
 16  20  24
 28  35  42
running:
 18  23  30
 22  28  36
 37  47  60


<br>

and potentially a little faster :)

In [10]:
# @coppertop(style=binary, module='notes.intuitive')
@coppertop(style=binary)
def mulMatMat4(A:Matrix, B:Matrix) -> Matrix:
    mA, nA = A >> shape
    mB, nB = B >> shape
    assert nA == mB
    answer = Matrix(mA, nB) | +(mutable&maddResult)                                # answer can be destructively updated
    for i in nA >> offset.sequence_:
        'outer:' >> PP
        outerProduct = (A >> atCol >> i) >> outerProductCR_ >> (B >> atRow >> i) >> PP  # use deferred outerProductCR_ to remove an allocation
        'running:' >> PP
        answer = answer >> madd >> outerProduct >> PP
    return answer | -(mutable&maddResult)

In [11]:
A >> mulMatMat4 >> B;

outer:
running:
  6  8  12
  6  8  12
  9  12  18
outer:
running:
 18  23  30
 22  28  36
 37  47  60


In [12]:
outerProductCR_(_,_)